In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from phi import math, field
from phi.math import wrap, channel, spatial, instance
from phi.field import SoftGeometryMask, AngularVelocity, Grid, divergence, spatial_gradient, where, CenteredGrid, PointCloud
from phi.geom import union, Geometry

In [3]:
data = math.random_uniform(spatial(x=10, y=10, z=10), channel(vector='x,y,z'))

In [4]:
data

(xˢ=10, yˢ=10, zˢ=10, vectorᶜ=x,y,z) 0.503 ± 0.290 (3e-05...1e+00)

In [5]:
# Extracting an instance from a Tensor
data.native(['x', 'y', 'z', 'vector'])

array([[[[0.4254249 , 0.45919806, 0.9591326 ],
         [0.32374433, 0.8585425 , 0.04971628],
         [0.9202786 , 0.79672027, 0.9890029 ],
         ...,
         [0.8599671 , 0.3286496 , 0.57626307],
         [0.10357234, 0.77478534, 0.31924862],
         [0.3619732 , 0.89294326, 0.11930441]],

        [[0.06041207, 0.30682352, 0.76435274],
         [0.695414  , 0.82507443, 0.10422496],
         [0.6765138 , 0.16482802, 0.7170042 ],
         ...,
         [0.5299192 , 0.54669785, 0.06430988],
         [0.56820655, 0.79208815, 0.59750247],
         [0.78726   , 0.2734776 , 0.9947038 ]],

        [[0.37104213, 0.94117564, 0.48121414],
         [0.48756835, 0.8649911 , 0.7409574 ],
         [0.27083138, 0.8272168 , 0.4740164 ],
         ...,
         [0.696854  , 0.47170344, 0.52053124],
         [0.08879066, 0.20126168, 0.87278885],
         [0.7491208 , 0.08310553, 0.5556203 ]],

        ...,

        [[0.0768082 , 0.966777  , 0.2965328 ],
         [0.86758333, 0.711357  , 0.02549846]

In [6]:
data *= 100

In [7]:
from phi.math._tensors import NativeTensor, TensorDim
vec: TensorDim = data.vector

In [8]:
# Packing the spatial dimensions into a single dimension
math.pack_dims(data, spatial, instance('spatial'))

(spatialⁱ=1000, vectorᶜ=x,y,z) 50.315 ± 28.998 (3e-03...1e+02)

In [9]:
vec['x', 'y', 'z']

(xˢ=10, yˢ=10, zˢ=10, vectorᶜ=x,y,z) 50.315 ± 28.998 (3e-03...1e+02)

In [10]:
from phi.field import CenteredGrid
from phi.geom import Box
lower = math.tensor([0, 0])
cg = CenteredGrid([0, 0], bounds=Box(lower=math.tensor([0, 0]), upper=math.tensor([10, 10])), resolution=10)

AssertionError: vector dimension of lower must list spatial dimension order